In [1]:
import pandas as pd
import numpy as np
import re
import os
import sys
import csv
import json
from datetime import datetime
from pathlib import Path

df = pd.read_excel('rawdata_blessings.xlsx', sheet_name='blessings')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 57 columns):
 #   Column                                                                                                                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                      --------------  -----  
 0   Response ID                                                                                                                                                                                                                                 415 non-null    int64  
 1   Gender                                                                                         

In [2]:
df.columns = df.columns.str.strip()
df.columns

Index(['Response ID', 'Gender', 'Age_range', 'Ethnicity', 'State',
       'Avg_monthly_income',
       'Anchor Smooth:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'Asahi:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'Carlsberg:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'Carlsberg Smooth Draught:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'C

In [3]:
df['Avg_monthly_income'] = df['Avg_monthly_income'].astype(str).replace({
    '1': 'Below RM2500',
    '2': 'Below RM2500',
    '3': 'Below RM2500',
    '4': 'Below RM2500',
    '5': 'RM2500 – RM4999',
    '6': 'RM2500 – RM4999',
    '7': 'RM2500 – RM4999',
    '8': 'RM5000 – RM6999',
    '9': 'RM5000 – RM6999',
    '10': 'RM7000 – RM10999',
    '11': 'RM7000 – RM10999',
    '12': 'RM7000 – RM10999',
    '13': 'RM7000 – RM10999',
})

In [4]:
print(df['Avg_monthly_income'].value_counts())
print('--------------')
print(df['Avg_monthly_income'].isna().sum()) # No missing values

Avg_monthly_income
RM2500 – RM4999     155
RM5000 – RM6999     111
RM7000 – RM10999     83
Below RM2500         66
Name: count, dtype: int64
--------------
0


In [5]:
df.columns

Index(['Response ID', 'Gender', 'Age_range', 'Ethnicity', 'State',
       'Avg_monthly_income',
       'Anchor Smooth:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'Asahi:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'Carlsberg:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'Carlsberg Smooth Draught:Here is a list of different brands of beer and stout in Malaysia. Please select all brands that you are aware of. Please also include all the brands that you have mentioned in the previous questions.',
       'C

In [6]:
brand_mapping = {
    "Anchor Smooth": "Anchor Smooth Brand",
    "Asahi": "Asahi Brand",
    "Carlsberg": "Carlsberg Brand",
    "Carlsberg Smooth Draught": "Carlsberg Smooth Draught Brand",
    "Carlsberg Special Brew": "Carlsberg Special Brew Brand",
    "Connor's Stout": "Connor's Stout Brand",
    "Danish Royal Stout": "Danish Royal Stout Brand",
    "Guinness Draught": "Guinness Draught Brand",
    "Guinness Foreign Extra Stout (FES)": "Guinness Foreign Extra Stout (FES) Brand",
    "Heineken 0.0": "Heineken 0.0 Brand",
    "Heineken Original": "Heineken Original Brand",
    "Kronenbourg 1664 Blanc": "Kronenbourg 1664 Blanc Brand",
    "Kronenbourg 1664 Rose": "Kronenbourg 1664 Rose Brand",
    "Skol": "Skol Brand",
    "Tiger Crystal": "Tiger Crystal Brand",
    "Tiger": "Tiger Brand",
    "Edelweiss": "Edelweiss Brand"
}

for keyword, new_name in brand_mapping.items():
    for col in df.columns:
        if keyword in col and "aware" in col:
            df = df.rename(columns={col: new_name})
            break  # stop once found

In [7]:
brand_mapping = {
    "Anchor Smooth": "Anchor Smooth Current",
    "Asahi": "Asahi Current",
    "Carlsberg": "Carlsberg Current",
    "Carlsberg Smooth Draught": "Carlsberg Smooth Draught Current",
    "Carlsberg Special Brew": "Carlsberg Special Brew Current",
    "Connor's Stout": "Connor's Stout Current",
    "Danish Royal Stout": "Danish Royal Stout Current",
    "Guinness Draught": "Guinness Draught Current",
    "Guinness Foreign Extra Stout (FES)": "Guinness Foreign Extra Stout (FES) Current",
    "Heineken 0.0": "Heineken 0.0 Current",
    "Heineken Original": "Heineken Original Current",
    "Kronenbourg 1664 Blanc": "Kronenbourg 1664 Blanc Current",
    "Kronenbourg 1664 Rose": "Kronenbourg 1664 Rose Current",
    "Skol": "Skol Current",
    "Tiger Crystal": "Tiger Crystal Current",
    "Tiger": "Tiger Current",
    "Edelweiss": "Edelweiss Current"
}

for keyword, new_name in brand_mapping.items():
    for col in df.columns:
        if keyword in col and "drank" in col:
            df = df.rename(columns={col: new_name})
            break  # stop once found

In [8]:
brand_mapping = {
    "Anchor Smooth": "Anchor Smooth Purchase Likelihood",
    "Asahi": "Asahi Purchase Likelihood",
    "Carlsberg": "Carlsberg Purchase Likelihood",
    "Carlsberg Smooth Draught": "Carlsberg Smooth Draught Purchase Likelihood",
    "Carlsberg Special Brew": "Carlsberg Special Brew Purchase Likelihood",
    "Connor's Stout": "Connor's Stout Purchase Likelihood",
    "Danish Royal Stout": "Danish Royal Stout Purchase Likelihood",
    "Guinness Draught": "Guinness Draught Purchase Likelihood",
    "Guinness Foreign Extra Stout (FES)": "Guinness Foreign Extra Stout (FES) Purchase Likelihood",
    "Heineken 0.0": "Heineken 0.0 Purchase Likelihood",
    "Heineken Original": "Heineken Original Purchase Likelihood",
    "Kronenbourg 1664 Blanc": "Kronenbourg 1664 Blanc Purchase Likelihood",
    "Kronenbourg 1664 Rose": "Kronenbourg 1664 Rose Purchase Likelihood",
    "Skol": "Skol Purchase Likelihood",
    "Tiger Crystal": "Tiger Crystal Purchase Likelihood",
    "Tiger": "Tiger Purchase Likelihood",
    "Edelweiss": "Edelweiss Purchase Likelihood"
}

for keyword, new_name in brand_mapping.items():
    for col in df.columns:
        if keyword in col and "drunk" in col:
            df = df.rename(columns={col: new_name})
            break  # stop once found

In [9]:
df.columns

Index(['Response ID', 'Gender', 'Age_range', 'Ethnicity', 'State',
       'Avg_monthly_income', 'Anchor Smooth Brand', 'Asahi Brand',
       'Carlsberg Brand', 'Carlsberg Smooth Draught Brand',
       'Carlsberg Special Brew Brand', 'Connor's Stout Brand',
       'Danish Royal Stout Brand', 'Guinness Draught Brand',
       'Guinness Foreign Extra Stout (FES) Brand', 'Heineken 0.0 Brand',
       'Heineken Original Brand', 'Kronenbourg 1664 Blanc Brand',
       'Kronenbourg 1664 Rose Brand', 'Skol Brand', 'Tiger Crystal Brand',
       'Tiger Brand', 'Edelweiss Brand', 'Anchor Smooth Current',
       'Asahi Current', 'Carlsberg Current',
       'Carlsberg Smooth Draught Current', 'Carlsberg Special Brew Current',
       'Connor's Stout Current', 'Danish Royal Stout Current',
       'Guinness Draught Current',
       'Guinness Foreign Extra Stout (FES) Current', 'Heineken 0.0 Current',
       'Heineken Original Current', 'Kronenbourg 1664 Blanc Current',
       'Kronenbourg 1664 Rose Curre

### Pivoting

In [10]:
columns_to_process = df.columns.tolist()
columns_to_process

['Response ID',
 'Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Brand',
 'Asahi Brand',
 'Carlsberg Brand',
 'Carlsberg Smooth Draught Brand',
 'Carlsberg Special Brew Brand',
 "Connor's Stout Brand",
 'Danish Royal Stout Brand',
 'Guinness Draught Brand',
 'Guinness Foreign Extra Stout (FES) Brand',
 'Heineken 0.0 Brand',
 'Heineken Original Brand',
 'Kronenbourg 1664 Blanc Brand',
 'Kronenbourg 1664 Rose Brand',
 'Skol Brand',
 'Tiger Crystal Brand',
 'Tiger Brand',
 'Edelweiss Brand',
 'Anchor Smooth Current',
 'Asahi Current',
 'Carlsberg Current',
 'Carlsberg Smooth Draught Current',
 'Carlsberg Special Brew Current',
 "Connor's Stout Current",
 'Danish Royal Stout Current',
 'Guinness Draught Current',
 'Guinness Foreign Extra Stout (FES) Current',
 'Heineken 0.0 Current',
 'Heineken Original Current',
 'Kronenbourg 1664 Blanc Current',
 'Kronenbourg 1664 Rose Current',
 'Skol Current',
 'Tiger Crystal Current',
 'Tiger Current',
 'Edelweis

In [11]:
# Include Response ID for count
columns_to_process = ['Response ID', 'Gender', 'Age_range', 'Ethnicity','State','Avg_monthly_income'] + columns_to_process

# Melt the DataFrame for processing
melted_df = df.melt(id_vars=['Gender', 'Age_range','Ethnicity','State','Avg_monthly_income'], value_vars=columns_to_process, 
                    var_name='Category', value_name='Value')

# Generate a pivot table without grand totals
pivot_table = pd.pivot_table(
    melted_df,
    values='Value',
    index=['Gender', 'Age_range','Ethnicity','State','Avg_monthly_income'],
    columns='Category',
    aggfunc='count',
    fill_value=0
)

# Reset index for cleaner output (optional)
pivot_table.reset_index(inplace=True)
print(pivot_table)

Category  Gender     Age_range Ethnicity     State Avg_monthly_income  \
0         Female         18-25   Chinese     Johor    RM2500 – RM4999   
1         Female         18-25   Chinese     Kedah       Below RM2500   
2         Female         18-25   Chinese    Melaka       Below RM2500   
3         Female         18-25   Chinese    Melaka    RM2500 – RM4999   
4         Female         18-25   Chinese    Melaka   RM7000 – RM10999   
..           ...           ...       ...       ...                ...   
122         Male  46 and above   Chinese     Perak    RM2500 – RM4999   
123         Male  46 and above   Chinese  Selangor    RM5000 – RM6999   
124         Male  46 and above   Chinese  Selangor   RM7000 – RM10999   
125         Male  46 and above   Chinese        WP    RM2500 – RM4999   
126         Male  46 and above   Chinese        WP    RM5000 – RM6999   

Category  Anchor Smooth Brand  Anchor Smooth Current  \
0                           1                      0   
1          

In [12]:
pivot_table.to_csv('checkcount_blessings.csv')

In [13]:
total_rows = len(df)

# List of columns to process for current usage
awareness_columns = [col for col in df.columns if "Brand" in col]

# Loop through each column to calculate awareness and unawareness percentages
for column in awareness_columns:
    pivot_table[f'Unaware of {column}'] = pivot_table['Response ID'] - pivot_table[column]
    pivot_table[f'{column} Awareness (%)'] = (pivot_table[column] / total_rows) * 100
    pivot_table[f'{column} Unawareness (%)'] = (pivot_table[f'Unaware of {column}'] / total_rows) * 100

    # Round the percentages to 1 decimal place
    pivot_table[f'{column} Awareness (%)'] = pivot_table[f'{column} Awareness (%)']
    pivot_table[f'{column} Unawareness (%)'] = pivot_table[f'{column} Unawareness (%)']

# Drop the original columns after processing
pivot_table.drop(columns=awareness_columns, inplace=True)

# Drop columns starting with "Unaware of"
unaware_columns = [col for col in pivot_table.columns if col.startswith('Unaware of')]
pivot_table.drop(columns=unaware_columns, inplace=True)

# Define the columns for display, ensuring awareness and unawareness are side by side
display_columns1 = ['Gender', 'Age_range','Ethnicity','State','Avg_monthly_income']
for column in awareness_columns:
    display_columns1.extend([f'{column} Awareness (%)', f'{column} Unawareness (%)'])

print(pivot_table[display_columns1])

Category  Gender     Age_range Ethnicity     State Avg_monthly_income  \
0         Female         18-25   Chinese     Johor    RM2500 – RM4999   
1         Female         18-25   Chinese     Kedah       Below RM2500   
2         Female         18-25   Chinese    Melaka       Below RM2500   
3         Female         18-25   Chinese    Melaka    RM2500 – RM4999   
4         Female         18-25   Chinese    Melaka   RM7000 – RM10999   
..           ...           ...       ...       ...                ...   
122         Male  46 and above   Chinese     Perak    RM2500 – RM4999   
123         Male  46 and above   Chinese  Selangor    RM5000 – RM6999   
124         Male  46 and above   Chinese  Selangor   RM7000 – RM10999   
125         Male  46 and above   Chinese        WP    RM2500 – RM4999   
126         Male  46 and above   Chinese        WP    RM5000 – RM6999   

Category  Anchor Smooth Brand Awareness (%)  \
0                                  0.240964   
1                            

In [14]:
total_rows = len(df)

# List of columns to process for current usage
usage_columns = [col for col in df.columns if "Current" in col]

# Loop through each column to calculate usage and non-usage percentages
for column in usage_columns:
    if column in pivot_table.columns:
        pivot_table[f'Not Using {column}'] = pivot_table['Response ID'] - pivot_table[column]
        pivot_table[f'{column} Usage (%)'] = (pivot_table[column] / total_rows) * 100
        pivot_table[f'{column} Non-usage (%)'] = (pivot_table[f'Not Using {column}'] / total_rows) * 100

        # Round the percentages to 1 decimal place
        pivot_table[f'{column} Usage (%)'] = pivot_table[f'{column} Usage (%)']
        pivot_table[f'{column} Non-usage (%)'] = pivot_table[f'{column} Non-usage (%)']
    else:
        print(f"Column '{column}' is missing in pivt_table.")

# Drop the original columns after processing
pivot_table.drop(columns=[col for col in usage_columns if col in pivot_table.columns], inplace=True)

# Drop columns starting with "Not Using"
not_using_columns = [col for col in pivot_table.columns if col.startswith('Not Using')]
pivot_table.drop(columns=not_using_columns, inplace=True)

# Define the columns for display, ensuring usage and non-usage are side by side
display_columns = ['Gender', 'Age_range', 'Ethnicity','State','Avg_monthly_income']
for column in usage_columns:
    if f'{column} Usage (%)' in pivot_table.columns and f'{column} Non-usage (%)' in pivot_table.columns:
        display_columns.extend([f'{column} Usage (%)', f'{column} Non-usage (%)'])

print(pivot_table[display_columns])

Category  Gender     Age_range Ethnicity     State Avg_monthly_income  \
0         Female         18-25   Chinese     Johor    RM2500 – RM4999   
1         Female         18-25   Chinese     Kedah       Below RM2500   
2         Female         18-25   Chinese    Melaka       Below RM2500   
3         Female         18-25   Chinese    Melaka    RM2500 – RM4999   
4         Female         18-25   Chinese    Melaka   RM7000 – RM10999   
..           ...           ...       ...       ...                ...   
122         Male  46 and above   Chinese     Perak    RM2500 – RM4999   
123         Male  46 and above   Chinese  Selangor    RM5000 – RM6999   
124         Male  46 and above   Chinese  Selangor   RM7000 – RM10999   
125         Male  46 and above   Chinese        WP    RM2500 – RM4999   
126         Male  46 and above   Chinese        WP    RM5000 – RM6999   

Category  Anchor Smooth Current Usage (%)  \
0                                     0.0   
1                                

In [15]:
total_rows = len(df)

# List of columns to process for current usage
purchase_columns = [col for col in df.columns if "Likelihood" in col]

# Store new columns in a dict first
new_columns = {}

# Loop through each column to calculate usage and non-usage percentages
for column in purchase_columns:
    if column in pivot_table.columns:
        new_columns[f'Not purchasing {column}'] = pivot_table['Response ID'] - pivot_table[column]
        new_columns[f'{column} (Likely)'] = ((pivot_table[column] / total_rows) * 100)
        new_columns[f'{column} (Unlikely)'] = ((new_columns[f'Not purchasing {column}'] / total_rows) * 100)
    else:
        print(f"Column '{column}' is missing in pivot_table.")

# Concatenate all new columns at once to avoid fragmentation
pivot_table = pd.concat([pivot_table, pd.DataFrame(new_columns)], axis=1)

# Drop the original columns after processing
pivot_table.drop(columns=[col for col in purchase_columns if col in pivot_table.columns], inplace=True)

# Drop columns starting with "Not Using"
not_purchasing_columns = [col for col in pivot_table.columns if col.startswith('Not purchasing')]
pivot_table.drop(columns=not_purchasing_columns, inplace=True)

# Define the columns for display, ensuring usage and non-usage are side by side
display_columns = ['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income']
for column in purchase_columns:
    likely_col = f'{column} (Likely)'
    unlikely_col = f'{column} (Unlikely)'
    if likely_col in pivot_table.columns and unlikely_col in pivot_table.columns:
        display_columns.extend([likely_col, unlikely_col])

print(pivot_table[display_columns])

     Gender     Age_range Ethnicity     State Avg_monthly_income  \
0    Female         18-25   Chinese     Johor    RM2500 – RM4999   
1    Female         18-25   Chinese     Kedah       Below RM2500   
2    Female         18-25   Chinese    Melaka       Below RM2500   
3    Female         18-25   Chinese    Melaka    RM2500 – RM4999   
4    Female         18-25   Chinese    Melaka   RM7000 – RM10999   
..      ...           ...       ...       ...                ...   
122    Male  46 and above   Chinese     Perak    RM2500 – RM4999   
123    Male  46 and above   Chinese  Selangor    RM5000 – RM6999   
124    Male  46 and above   Chinese  Selangor   RM7000 – RM10999   
125    Male  46 and above   Chinese        WP    RM2500 – RM4999   
126    Male  46 and above   Chinese        WP    RM5000 – RM6999   

     Anchor Smooth Purchase Likelihood (Likely)  \
0                                      0.000000   
1                                      0.000000   
2                             

In [16]:
# total_rows = len(df)

# # List of columns to process for purchase likelihood
# purchase_columns = [col for col in df.columns if "Purchase" in col]

# # Loop through each column to calculate likely and unlikely percentages
# for column in purchase_columns:
#     if column in df.columns:
#         # Group and sum the purchase likelihood count per demographic group
#         purchase_counts = df.groupby(['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income'])[column].sum().reset_index()

#         # Rename the column for clarity
#         purchase_counts.rename(columns={column: f'{column}_count'}, inplace=True)

#         # Merge into pivot_table
#         pivot_table = pivot_table.merge(purchase_counts, on=['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income'], how='left')

#         # Fill NaNs with 0
#         pivot_table[f'{column}_count'] = pivot_table[f'{column}_count'].fillna(0)

#         # Calculate percentages
#         pivot_table[f'{column} (Likely)'] = (pivot_table[f'{column}_count'] / total_rows) * 100
#         pivot_table[f'{column} (Unlikely)'] = ((pivot_table['Response ID'] - pivot_table[f'{column}_count']) / total_rows) * 100

#         # Round the results
#         pivot_table[f'{column} (Likely)'] = pivot_table[f'{column} (Likely)'].round(2)
#         pivot_table[f'{column} (Unlikely)'] = pivot_table[f'{column} (Unlikely)'].round(2)
#     else:
#         print(f"Column '{column}' is missing in df.")

# # Drop intermediate count columns
# intermediate_columns = [f'{col}_count' for col in purchase_columns if f'{col}_count' in pivot_table.columns]
# pivot_table.drop(columns=intermediate_columns, inplace=True)

# # Define display columns
# display_columns = ['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income']
# for column in purchase_columns:
#     if f'{column} (Likely)' in pivot_table.columns and f'{column} (Unlikely)' in pivot_table.columns:
#         display_columns.extend([f'{column} (Likely)', f'{column} (Unlikely)'])

# print(pivot_table[display_columns])

In [17]:
pivot_table.shape

(127, 108)

In [18]:
pivot_table = pivot_table.drop(columns=['Response ID'])
pivot_table.columns

Index(['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income',
       'Anchor Smooth Brand Awareness (%)',
       'Anchor Smooth Brand Unawareness (%)', 'Asahi Brand Awareness (%)',
       'Asahi Brand Unawareness (%)', 'Carlsberg Brand Awareness (%)',
       ...
       'Kronenbourg 1664 Rose Purchase Likelihood (Likely)',
       'Kronenbourg 1664 Rose Purchase Likelihood (Unlikely)',
       'Skol Purchase Likelihood (Likely)',
       'Skol Purchase Likelihood (Unlikely)',
       'Tiger Crystal Purchase Likelihood (Likely)',
       'Tiger Crystal Purchase Likelihood (Unlikely)',
       'Tiger Purchase Likelihood (Likely)',
       'Tiger Purchase Likelihood (Unlikely)',
       'Edelweiss Purchase Likelihood (Likely)',
       'Edelweiss Purchase Likelihood (Unlikely)'],
      dtype='object', length=107)

In [19]:
merged_blessings = pivot_table

merged_blessings.to_csv('pivot-blessings.csv',float_format='%.1f', index=False)
merged_blessings.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Columns: 107 entries, Gender to Edelweiss Purchase Likelihood (Unlikely)
dtypes: float64(102), object(5)
memory usage: 106.3+ KB


### Preparing each df for each sheets in xlsx

In [20]:
columns_to_process1 = merged_blessings.columns[0:5].tolist() + [col for col in merged_blessings.columns if "Current Usage (%)" in col]
columns_to_process1

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Current Usage (%)',
 'Asahi Current Usage (%)',
 'Carlsberg Current Usage (%)',
 'Carlsberg Smooth Draught Current Usage (%)',
 'Carlsberg Special Brew Current Usage (%)',
 "Connor's Stout Current Usage (%)",
 'Danish Royal Stout Current Usage (%)',
 'Guinness Draught Current Usage (%)',
 'Guinness Foreign Extra Stout (FES) Current Usage (%)',
 'Heineken 0.0 Current Usage (%)',
 'Heineken Original Current Usage (%)',
 'Kronenbourg 1664 Blanc Current Usage (%)',
 'Kronenbourg 1664 Rose Current Usage (%)',
 'Skol Current Usage (%)',
 'Tiger Crystal Current Usage (%)',
 'Tiger Current Usage (%)',
 'Edelweiss Current Usage (%)']

In [21]:
columns_to_process2 = merged_blessings.columns[0:5].tolist() + [col for col in merged_blessings.columns if "Current Non-usage (%)" in col]
columns_to_process2

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Current Non-usage (%)',
 'Asahi Current Non-usage (%)',
 'Carlsberg Current Non-usage (%)',
 'Carlsberg Smooth Draught Current Non-usage (%)',
 'Carlsberg Special Brew Current Non-usage (%)',
 "Connor's Stout Current Non-usage (%)",
 'Danish Royal Stout Current Non-usage (%)',
 'Guinness Draught Current Non-usage (%)',
 'Guinness Foreign Extra Stout (FES) Current Non-usage (%)',
 'Heineken 0.0 Current Non-usage (%)',
 'Heineken Original Current Non-usage (%)',
 'Kronenbourg 1664 Blanc Current Non-usage (%)',
 'Kronenbourg 1664 Rose Current Non-usage (%)',
 'Skol Current Non-usage (%)',
 'Tiger Crystal Current Non-usage (%)',
 'Tiger Current Non-usage (%)',
 'Edelweiss Current Non-usage (%)']

In [22]:
columns_to_process3 = merged_blessings.columns[0:5].tolist() + [col for col in merged_blessings.columns if "Purchase Likelihood (Likely)" in col]
columns_to_process3

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Purchase Likelihood (Likely)',
 'Asahi Purchase Likelihood (Likely)',
 'Carlsberg Purchase Likelihood (Likely)',
 'Carlsberg Smooth Draught Purchase Likelihood (Likely)',
 'Carlsberg Special Brew Purchase Likelihood (Likely)',
 "Connor's Stout Purchase Likelihood (Likely)",
 'Danish Royal Stout Purchase Likelihood (Likely)',
 'Guinness Draught Purchase Likelihood (Likely)',
 'Guinness Foreign Extra Stout (FES) Purchase Likelihood (Likely)',
 'Heineken 0.0 Purchase Likelihood (Likely)',
 'Heineken Original Purchase Likelihood (Likely)',
 'Kronenbourg 1664 Blanc Purchase Likelihood (Likely)',
 'Kronenbourg 1664 Rose Purchase Likelihood (Likely)',
 'Skol Purchase Likelihood (Likely)',
 'Tiger Crystal Purchase Likelihood (Likely)',
 'Tiger Purchase Likelihood (Likely)',
 'Edelweiss Purchase Likelihood (Likely)']

In [23]:
columns_to_process4 = merged_blessings.columns[0:5].tolist() + [col for col in merged_blessings.columns if "Purchase Likelihood (Unlikely)" in col]
columns_to_process4

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Purchase Likelihood (Unlikely)',
 'Asahi Purchase Likelihood (Unlikely)',
 'Carlsberg Purchase Likelihood (Unlikely)',
 'Carlsberg Smooth Draught Purchase Likelihood (Unlikely)',
 'Carlsberg Special Brew Purchase Likelihood (Unlikely)',
 "Connor's Stout Purchase Likelihood (Unlikely)",
 'Danish Royal Stout Purchase Likelihood (Unlikely)',
 'Guinness Draught Purchase Likelihood (Unlikely)',
 'Guinness Foreign Extra Stout (FES) Purchase Likelihood (Unlikely)',
 'Heineken 0.0 Purchase Likelihood (Unlikely)',
 'Heineken Original Purchase Likelihood (Unlikely)',
 'Kronenbourg 1664 Blanc Purchase Likelihood (Unlikely)',
 'Kronenbourg 1664 Rose Purchase Likelihood (Unlikely)',
 'Skol Purchase Likelihood (Unlikely)',
 'Tiger Crystal Purchase Likelihood (Unlikely)',
 'Tiger Purchase Likelihood (Unlikely)',
 'Edelweiss Purchase Likelihood (Unlikely)']

In [24]:
columns_to_process5 = merged_blessings.columns[0:5].tolist() + [col for col in merged_blessings.columns if "Awareness" in col]
columns_to_process5

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Brand Awareness (%)',
 'Asahi Brand Awareness (%)',
 'Carlsberg Brand Awareness (%)',
 'Carlsberg Smooth Draught Brand Awareness (%)',
 'Carlsberg Special Brew Brand Awareness (%)',
 "Connor's Stout Brand Awareness (%)",
 'Danish Royal Stout Brand Awareness (%)',
 'Guinness Draught Brand Awareness (%)',
 'Guinness Foreign Extra Stout (FES) Brand Awareness (%)',
 'Heineken 0.0 Brand Awareness (%)',
 'Heineken Original Brand Awareness (%)',
 'Kronenbourg 1664 Blanc Brand Awareness (%)',
 'Kronenbourg 1664 Rose Brand Awareness (%)',
 'Skol Brand Awareness (%)',
 'Tiger Crystal Brand Awareness (%)',
 'Tiger Brand Awareness (%)',
 'Edelweiss Brand Awareness (%)']

In [25]:
columns_to_process6 = merged_blessings.columns[0:5].tolist() + [col for col in merged_blessings.columns if "Unawareness" in col]
columns_to_process6

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Anchor Smooth Brand Unawareness (%)',
 'Asahi Brand Unawareness (%)',
 'Carlsberg Brand Unawareness (%)',
 'Carlsberg Smooth Draught Brand Unawareness (%)',
 'Carlsberg Special Brew Brand Unawareness (%)',
 "Connor's Stout Brand Unawareness (%)",
 'Danish Royal Stout Brand Unawareness (%)',
 'Guinness Draught Brand Unawareness (%)',
 'Guinness Foreign Extra Stout (FES) Brand Unawareness (%)',
 'Heineken 0.0 Brand Unawareness (%)',
 'Heineken Original Brand Unawareness (%)',
 'Kronenbourg 1664 Blanc Brand Unawareness (%)',
 'Kronenbourg 1664 Rose Brand Unawareness (%)',
 'Skol Brand Unawareness (%)',
 'Tiger Crystal Brand Unawareness (%)',
 'Tiger Brand Unawareness (%)',
 'Edelweiss Brand Unawareness (%)']

In [26]:
# Create DataFrames for each set of columns
df1 = merged_blessings[columns_to_process1]
df2 = merged_blessings[columns_to_process2]
df3 = merged_blessings[columns_to_process3]
df4 = merged_blessings[columns_to_process4]
df5 = merged_blessings[columns_to_process5]
df6 = merged_blessings[columns_to_process6]

In [27]:
# Define a list of tuples with the dataframe and the string to replace
dfs_info = [
    (df1, "Current Usage (%)"),
    (df2, "Current Non-usage (%)"),
    (df3, "Purchase Likelihood (Likely)"),
    (df4, "Purchase Likelihood (Unlikely)"),
    (df5, "Brand Awareness (%)"),
    (df6, "Brand Unawareness (%)")
]

# List to store the transformed DataFrames
melted_dfs = []

# Loop through each and apply the same operation
for df, replace_str in dfs_info:
    melted = df.melt(
        id_vars=['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income'],
        var_name='Brand',
        value_name='Percentage'
    )
    melted['Brand'] = melted['Brand'].str.replace(replace_str, '', regex=False)
    melted_dfs.append(melted)

# Unpack the results back to original variables
df1, df2, df3, df4, df5, df6 = melted_dfs


In [28]:
# Dictionary to store DataFrames with their respective attributes
dataframes = {
    'Current Usage': df1,
    'Current Non-usage': df2,
    'Purchase Likelihood (Likely)': df3,
    'Purchase Likelihood (Unlikely)': df4,
    'Awareness': df5,
    'Unawareness': df6
}

def add_category_and_attribute(df, attribute_name):
    df['Category'] = 'Beverages'
    df['Country'] = 'Malaysia'
    df['Year'] = 2023
    df['Attributes'] = attribute_name
    return df

# Process all DataFrames and filter out empty ones
processed_dfs = [add_category_and_attribute(df, name) for name, df in dataframes.items() if df is not None and not df.empty]

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(processed_dfs, ignore_index=True) if processed_dfs else None

# Generate Composite Key after merging
if final_df is not None:
    # Ensure all columns in the composite key are strings
    final_df['Composite_Key'] = (
        final_df['Ethnicity'].astype(str) + '_' +
        # final_df['Brand'].astype(str) + '_' +
        final_df['Age_range'].astype(str) + '_' +
        final_df['Gender'].astype(str) + '_' +
        final_df['Avg_monthly_income'].astype(str) + '_' +
        final_df['State'].astype(str)
        # final_df['Category'].astype(str) + '_' +
        # final_df['Country'].astype(str) + '_' +
        # final_df['Year'].astype(str) + '_' +
        # final_df['Attributes'].astype(str)
    )



# # Define the output directory
# output_dir = Path(r"C:\Users\aisar\OneDrive\Documents\Mobile-Data-App\BLS\CLEANED")
# output_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

# # Define the file path
# output_file = output_dir / "maxis-final.xlsx"

# # Write to Excel
# final_df.to_excel(output_file, sheet_name='Maxis', float_format='%.1f', index=False)
# print(f"Excel file created successfully: {output_file}")

In [ ]:
# Remove rows where Percentage is 0
final_df = final_df[final_df['Percentage'] != 0]

final_df.to_excel('Blessings-final.xlsx', sheet_name='Blessings', index=False)
print("Excel file created successfully!")

Excel file created successfully!


In [30]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8586 entries, 9 to 12951
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              8586 non-null   object 
 1   Age_range           8586 non-null   object 
 2   Ethnicity           8586 non-null   object 
 3   State               8586 non-null   object 
 4   Avg_monthly_income  8586 non-null   object 
 5   Brand               8586 non-null   object 
 6   Percentage          8586 non-null   float64
 7   Category            8586 non-null   object 
 8   Country             8586 non-null   object 
 9   Year                8586 non-null   int64  
 10  Attributes          8586 non-null   object 
 11  Composite_Key       8586 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 872.0+ KB


In [31]:
final_df['Percentage'] = pd.to_numeric(final_df['Percentage'], errors='coerce')
attribute_percentages = final_df.groupby('Attributes')['Percentage'].sum().reset_index()
attribute_percentages['Percentage'] = attribute_percentages['Percentage'].round(1)
print(attribute_percentages)

                       Attributes  Percentage
0                       Awareness       582.7
1               Current Non-usage      1504.3
2                   Current Usage       195.7
3    Purchase Likelihood (Likely)       346.3
4  Purchase Likelihood (Unlikely)      1353.7
5                     Unawareness      1117.3


In [34]:
# Convert Percentage to numeric
final_df['Percentage'] = pd.to_numeric(final_df['Percentage'], errors='coerce')

# Create pivot table
pivot = final_df.pivot_table(
    index='Attributes',
    columns='Brand',
    values='Percentage',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total'
).round(1)

# Replace NaN with empty strings (optional)
pivot = pivot.fillna('')

# Rename columns/rows to match the example (if needed)
pivot = pivot.rename_axis('Row Labels', axis=0).rename_axis('Column Labels', axis=1)
pivot

Column Labels,Anchor Smooth,Asahi,Carlsberg,Carlsberg Smooth Draught,Carlsberg Special Brew,Connor's Stout,Danish Royal Stout,Edelweiss,Guinness Draught,Guinness Foreign Extra Stout (FES),Heineken 0.0,Heineken Original,Kronenbourg 1664 Blanc,Kronenbourg 1664 Rose,Skol,Tiger,Tiger Crystal,Grand Total
Row Labels,,,,,,,,,,,,,,,,,,
Awareness,30.4,38.1,53.5,33.7,29.6,31.6,29.2,32.0,32.5,24.3,34.7,44.6,22.2,21.2,34.5,54.5,36.1,582.7
Current Non-usage,92.0,90.6,74.9,92.0,93.0,90.4,91.3,92.5,90.4,92.8,88.7,80.5,93.5,95.9,87.5,69.2,89.2,1504.3
Current Usage,8.0,9.4,25.1,8.0,7.0,9.6,8.7,7.5,9.6,7.2,11.3,19.5,6.5,4.1,12.5,30.8,10.8,195.7
Purchase Likelihood (Likely),17.3,18.6,38.1,19.8,16.1,17.8,14.2,16.4,19.3,11.8,22.2,29.9,12.0,11.1,19.3,40.2,22.2,346.3
Purchase Likelihood (Unlikely),82.7,81.4,61.9,80.2,83.9,82.2,85.8,83.6,80.7,88.2,77.8,70.1,88.0,88.9,80.7,59.8,77.8,1353.7
Unawareness,69.6,61.9,46.5,66.3,70.4,68.4,70.8,68.0,67.5,75.7,65.3,55.4,77.8,78.8,65.5,45.5,63.9,1117.3
Grand Total,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,5100.0


In [33]:
import pandas as pd

# Ensure Percentage is numeric and handle NaNs
final_df['Percentage'] = pd.to_numeric(final_df['Percentage'], errors='coerce')

# Create pivot table with 0 for missing values
pivot = final_df.pivot_table(
    index='Attributes',
    columns='Brand',
    values='Percentage',
    aggfunc='sum',
    fill_value=0,  # Critical for matching Excel's behavior
    margins=True,
    margins_name='Grand Total'
).round(1)

# Reset index/columns for clarity
pivot = pivot.rename_axis('Row Labels', axis=0).rename_axis('Column Labels', axis=1)
print(pivot)

Column Labels                   Anchor Smooth   Asahi   Carlsberg   \
Row Labels                                                           
Awareness                                 30.4    38.1        53.5   
Current Non-usage                         92.0    90.6        74.9   
Current Usage                              8.0     9.4        25.1   
Purchase Likelihood (Likely)              17.3    18.6        38.1   
Purchase Likelihood (Unlikely)            82.7    81.4        61.9   
Unawareness                               69.6    61.9        46.5   
Grand Total                              300.0   300.0       300.0   

Column Labels                   Carlsberg Smooth Draught   \
Row Labels                                                  
Awareness                                            33.7   
Current Non-usage                                    92.0   
Current Usage                                         8.0   
Purchase Likelihood (Likely)                         19.8   
Pur